In [69]:
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from pathlib import Path
import pandas as pd

In [ ]:
#save_dir = '/home/jl277509/Documents/Ablation_2_models/translation.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/7_translation'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/trimdepth_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/8_trimdepth_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/trimextremities_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/9_trimextremities_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/cutin_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/10_cutin_translation_3'

save_dir = '/home/jl277509/Documents/Ablation_2_models/cutout_translation_3.csv'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models/11_cutout_translation_3'

save_dir = '/home/jl277509/Documents/Ablation_2_models/6_no_augm.csv'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models/11_cutout_translation_3'
n_models = 2

# Step 1 : recover all perf values (with cross val std)

In [85]:
sub_dirs = {'SOr': [f'{basedir}/SOr_left_UKB40/', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'cv_score', 'cv_stds', 'splits_score'], # NB : error in pipeline stds -> std ...
            'FIP': [f'{basedir}/FIP_right_UKB40/', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'cv_score', 'cv_std', 'splits_score'],
            'SC-sylv': [f'{basedir}/SC-sylv_left_UKB40/', [f'hcp_isomap_custom_embeddings/Isomap_central_left_dim{k}/test_values.json' for k in range(1,7)], 'cv_score', 'cv_std', 'splits_score']}

l = os.listdir(basedir)
root_names = [elem for elem in l if ('UKB40' not in elem) and '#' not in elem]

In [86]:
# store everything globally
l_model_glob = []
l_perf_glob = []
l_std_glob = []
l_splits_scores_glob = []
l_region_glob = []
l_task_glob = []


for region, (sub_dir, auc_files, score, std, split_score) in sub_dirs.items(): 
    for auc_file in auc_files:
        models = os.listdir(sub_dir)
        models = [elem for elem in models if 'config' not in elem and 'multirun' not in elem]
        models.sort()
        task = Path(auc_file).parts[-2]
        for name in root_names:
            #l_models = [elem for elem in models if elem.startswith(name)]
            l_models = [elem for elem in models if name in elem]
            l_perfs = []
            l_stds = []
            l_splits_scores = []
            for model in l_models:
                try:
                    with open(os.path.join(sub_dir, model, auc_file), 'r') as f:
                        perf = json.load(f)
                except Exception:
                    print(f'{os.path.join(sub_dir, model, auc_file)} not found') ## NB : can't continue if not all files found ...

                if isinstance(perf[score], float):
                    value = perf[score]
                else:
                    value = perf[score][0]
                l_perfs.append(value)
                if isinstance(perf[std], float):
                    value = perf[std]
                else:
                    value = perf[std][0]
                l_stds.append(value)
                value = perf[split_score] # is a list
                l_splits_scores.append(value)
                
            l_model_glob.append(l_models)
            l_perf_glob.append(l_perfs)
            l_std_glob.append(l_stds)
            l_splits_scores_glob.append(l_splits_scores)
            l_region_glob.append([region for _ in range(len(l_perfs))])
            l_task_glob.append([task for _ in range(len(l_perfs))])

In [87]:
# reformat lists and put inside DataFrame
def flatten_sum(matrix):
    return sum(matrix, [])
cols = ['model', 'region', 'task', 'score', 'std']
values = np.array([flatten_sum(l_model_glob), flatten_sum(l_region_glob), flatten_sum(l_task_glob), flatten_sum(l_perf_glob), flatten_sum(l_std_glob)]).T
df = pd.DataFrame(data=values, columns=cols)
df['splits_score']=flatten_sum(l_splits_scores_glob)

# Step 2 : reorganize the df with the parameters

Translation

In [ ]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'translation_1' in model:
        list_magnitude.append(1)
    elif 'translation_2' in model:
        list_magnitude.append(2)
    elif 'translation_3' in model:
        list_magnitude.append(3)
    elif 'translation_4' in model:
        list_magnitude.append(4)
    elif 'translation_5' in model:
        list_magnitude.append(5)
    elif 'translation_6' in model:
        list_magnitude.append(6)

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)
    elif 'p100' in model:
        list_proba.append(100) 

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

Trimdepth

In [ ]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_mode=[]
list_proba=[]
for model in df['model'].tolist():
    if '2mm' in model:
        list_magnitude.append(2)
    elif '4mm' in model:
        list_magnitude.append(4)
    elif '6mm' in model:
        list_magnitude.append(6)
    elif '8mm' in model:
        list_magnitude.append(8)

    if 'constant' in model:
        list_mode.append('constant')
    else:
        list_mode.append('per branch')

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

df['Magnitude']=list_magnitude
df['Mode']=list_mode
df['Proba'] = list_proba
parameters=['Magnitude', 'Mode', 'Proba']

Trimextremities

In [31]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'pepper50' in model:
        list_magnitude.append(50)
    elif 'pepper100' in model:
        list_magnitude.append(100)

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

cutin

In [75]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
list_bottom_proba=[]
for model in df['model'].tolist():
    if 'bottom50' in model:
        list_bottom_proba.append(50)
    elif 'bottom70' in model:
        list_bottom_proba.append(70)

    if '20_40' in model:
        list_magnitude.append('[20,40]')
    elif '30_50' in model:
        list_magnitude.append('[30,50]')
    elif '40_60' in model:
        list_magnitude.append('[40,60]')
    elif '50_70' in model:
        list_magnitude.append('[50,70]')

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
df['Bottom_Proba']=list_bottom_proba
parameters=['Magnitude', 'Bottom_Proba', 'Proba']

cutout

In [88]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
list_bottom_proba=[]
for model in df['model'].tolist():
    if 'bottom50' in model:
        list_bottom_proba.append(50)
    elif 'bottom70' in model:
        list_bottom_proba.append(70)

    if '5_15' in model:
        list_magnitude.append('[5,15]')
    elif '10_20' in model:
        list_magnitude.append('[10,20]')
    elif '15_25' in model:
        list_magnitude.append('[15,25]')
    elif '20_30' in model:
        list_magnitude.append('[20,30]')

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
df['Bottom_Proba']=list_bottom_proba
parameters=['Magnitude', 'Bottom_Proba', 'Proba']

In [89]:
df_reformat = df.copy()
df_reformat = df_reformat.drop_duplicates(['model', 'task']).reset_index(drop=True)
df_reformat = df_reformat.drop(labels='model', axis='columns')
df_reformat['training_number']=[k%n_models for k in range(len(df_reformat))]
parameters.append('training_number')

In [90]:
# Pivoting the table
df_pivot = df_reformat.pivot(index=parameters, columns=['task', 'region'], values=['score', 'std', 'splits_score'])
df_pivot = df_pivot.dropna()

# Renaming columns
df_pivot.columns = [f'score_{col}' for col in df_pivot.columns]

# Resetting index to get a flat DataFrame
df_pivot = df_pivot.reset_index()

isomap_score_cols = [elem for elem in df_pivot.columns if "score_('score', 'Isomap" in elem]
df_pivot['score_Isomap_left_global'] = df_pivot[isomap_score_cols].astype(float).mean(axis=1)

In [91]:
df_scores = df_pivot[["score_('score', 'Left_OFC', 'SOr')", "score_('score', 'Right_FIP', 'FIP')", "score_Isomap_left_global"]].apply(pd.to_numeric).mul(100).round(1)
df_scores.columns=['SOr', 'FIP', 'Isomap']

# get stds for Isomap
df_splits_score_isomap = df_pivot[[f"score_('splits_score', 'Isomap_central_left_dim{k}', 'SC-sylv')" for k in range(1,7)]]
avg_per_split = df_splits_score_isomap.apply(lambda row: np.mean(row.tolist(), axis=0), axis=1)
std_isomap = avg_per_split.apply(lambda row : np.std(row.tolist()))

# get the std df
df_stds = df_pivot[["score_('std', 'Left_OFC', 'SOr')", "score_('std', 'Right_FIP', 'FIP')"]]
df_stds['isomap']=std_isomap
df_stds = df_stds.apply(pd.to_numeric).mul(100).round(1)
df_stds.columns=['SOr', 'FIP', 'Isomap']

# combine in one df
formatted_df = df_scores.astype(str) + "(" + df_stds.astype(str) + ")"

formatted_df = pd.concat((df_pivot[parameters], formatted_df), axis=1)

/tmp/ipykernel_219180/354669320.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stds['isomap']=std_isomap


In [92]:
formatted_df

,Magnitude,Bottom_Proba,Proba,training_number,SOr,FIP,Isomap
0,"[10,20]",50,40,0,70.8(1.9),72.8(5.1),56.3(3.9)
1,"[10,20]",50,40,1,74.4(1.8),74.4(4.3),55.7(3.4)
2,"[10,20]",50,80,0,64.6(2.8),77.7(6.3),57.9(4.0)
3,"[10,20]",50,80,1,68.6(2.7),78.3(4.9),57.8(3.7)
4,"[10,20]",70,40,0,72.8(1.7),79.3(5.8),46.0(4.0)
5,"[10,20]",70,40,1,69.2(3.6),69.3(4.2),49.7(4.2)
6,"[10,20]",70,80,0,72.2(2.4),78.5(6.4),55.8(3.8)
7,"[10,20]",70,80,1,73.2(2.3),78.3(5.6),55.9(3.6)
8,"[15,25]",50,40,0,73.2(1.7),79.8(5.4),54.7(3.5)
9,"[15,25]",50,40,1,71.6(1.6),78.1(7.3),54.4(4.0)


# Save df

In [93]:
formatted_df.to_csv(save_dir, index=False)